# Animation

In [1]:
#include <xthreejs/xthreejs.hpp>

In [2]:
std::size_t view_width = 600;
std::size_t view_height = 400;

Let's first set up a basic scene with a cube and a sphere,

In [3]:
auto sphere = xthree::mesh_generator()
    .geometry(xthree::sphere_buffer_geometry_generator()
              .radius(1)
              .widthSegments(32)
              .heightSegments(16)
              .finalize()
              )
    .material(xthree::mesh_standard_material_generator()
              .color("red")
              .finalize()
              )
    .finalize();

In [4]:
auto cube = xthree::mesh_generator()
    .geometry(xthree::box_buffer_geometry_generator()
             .width(1)
             .height(1)
             .depth(1)
             .finalize()
             )
    .material(xthree::mesh_standard_material_generator()
             .color("green")
             .finalize()
             )
    .position({2, 0, 4})
    .finalize();

as well as lighting and camera:

In [5]:
auto camera = xthree::perspective_camera_generator()
    .position({10, 6, 10})
    .aspect(view_width/view_height)
    .finalize();
auto key_light = xthree::directional_light_generator()
    .position({0, 10, 10})
    .finalize();
auto ambient_light = xthree::ambient_light();

## Keyframe animation

The three.js animation system is built as a [keyframe](https://en.wikipedia.org/wiki/Key_frame) system. We'll demonstrate this by animating the position and rotation of our camera.

First, we set up the keyframes for the position and the rotation separately:

In [6]:
auto positon_track = xthree::vector_keyframe_track_generator()
    .name(".position")
    .times({0, 2, 5})
    .values({ 10, 6, 10,
             6.3, 3.78, 6.3,
            -2.98, 0.84, 9.2
            })
    .finalize();

In [7]:
auto rotation_track = xthree::quaternion_keyframe_track_generator()
    .name(".quaternion")
    .times({0, 2, 5})
    .values({-0.184, 0.375, 0.0762, 0.905,
            -0.184, 0.375, 0.0762, 0.905,
            -0.0430, -0.156, -0.00681, 0.987,
            })
    .finalize();

Next, we create an animation clip combining the two tracks, and finally an animation action to control the animation. See the three.js docs for more details on the different responsibilities of the different classes.

In [8]:
auto camera_clip = xthree::animation_clip_generator()
    .tracks({positon_track, rotation_track})
    .finalize();

In [9]:
auto camera_action = xthree::animation_action_generator()
    .mixer(xthree::animation_mixer_generator()
           .rootObject(camera)
           .finalize()
           )
    .clip(camera_clip)
    .localRoot(camera)
    .finalize();

Now, let's see it in action:

In [10]:
auto scene = xthree::scene_generator()
    .children({sphere, cube, camera, key_light, ambient_light})
    .finalize();
auto controller = xthree::orbit_controls_generator()
    .controlling(camera)
    .finalize();
auto renderer = xthree::renderer_generator()
    .camera(camera)
    .scene(scene)
    .controls({controller})
    ._width(view_width)
    ._height(view_height)
    .finalize();

In [11]:
renderer.display()

A Jupyter widget

In [12]:
camera_action.display();

A Jupyter widget

Let's add another animation clip, this time animating the color of the sphere's material:

In [11]:
color_track = ColorKeyframeTrack(name='.material.color',
    times=[0, 1], values=[1, 0, 0,  0, 0, 1])  # red to blue

color_clip = AnimationClip(tracks=[color_track], duration=1.5)
color_action = AnimationAction(AnimationMixer(sphere), color_clip, sphere)

/home/loic/miniconda3/envs/cling-benjamin/lib/python3.5/site-packages/ipydatawidgets/ndarray/serializers.py:62: UserWarning: Cannot serialize (u)int64 data, Javascript does not support it. Casting to (u)int32.
  warnings.warn('Cannot serialize (u)int64 data, Javascript does not support it. '


In [12]:
color_action

AnimationAction(clip=AnimationClip(duration=1.5, tracks=(ColorKeyframeTrack(name='.material.color', times=array([0, 1]), values=array([1, 0, 0, 0, 0, 1])),)), localRoot=Mesh(geometry=SphereBufferGeometry(heightSegments=16, type='SphereBufferGeometry', widthSegments=32), material=MeshStandardMaterial(alphaMap=None, aoMap=None, bumpMap=None, color='red', defines={'STANDARD': ''}, displacementMap=None, emissiveMap=None, envMap=None, lightMap=None, map=None, metalnessMap=None, normalMap=None, normalScale=(1.0, 1.0), roughnessMap=None, type='MeshStandardMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0)), mixer=AnimationMixer(rootObject=Mesh(geometry=SphereBufferGeometry(heightSegments=16, type='SphereBufferGeometry', widthSegments=32), material=MeshStandardMaterial(alphaMap=None, aoMap=None, bumpMap=None, color='red', defines={'STANDARD': ''}, displacementMap=None, emissiveMap=None, envMap=None, lightMap=None, map=None, metalnessMap=None, normalMap=None, normalScale=(1.0, 1.0), roughnessMap=None, type='MeshStandardMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0))))

Note how the two animation clips can freely be combined since they affect different properties. It's also worth noting that the color animation can be combined with manual camera control, while the camera animation cannot. When animating the camera, you might want to consider disabling the manual controls.

### Animating rotation

When animating the camera rotation above, we used the camera's `quaternion`. This is the most robust method if animating free-form rotations. For example, the animation above was created by first moving the camera manually, and then reading out its `position` and `quaternion` properties at the wanted views. If you want more intuitive axes control, it is possible to animate the `rotation` sub-attributes instead, as shown below.

In [13]:
f = """
function f(origu,origv) {
    // scale u and v to the ranges I want: [0, 2*pi]
    var u = 2*Math.PI*origu;
    var v = 2*Math.PI*origv;
    
    var x = Math.sin(u);
    var y = Math.cos(v);
    var z = Math.cos(u+v);
    
    return new THREE.Vector3(x,y,z)
}
"""
surf_g = ParametricGeometry(func=f, slices=16, stacks=16);

surf1 = Mesh(geometry=surf_g, material=MeshLambertMaterial(color='green', side='FrontSide'))
surf2 = Mesh(geometry=surf_g, material=MeshLambertMaterial(color='yellow', side='BackSide'))
surf = Group(children=[surf1, surf2])

camera2 = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
scene2 = Scene(children=[surf, camera2,
                         DirectionalLight(position=[3, 5, 1], intensity=0.6),
                         AmbientLight(intensity=0.5)])
renderer2 = Renderer(camera=camera2, scene=scene2, controls=[OrbitControls(controlling=camera2)],
                     width=view_width, height=view_height)
display(renderer2)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), controls=[OrbitControls(controlling=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)))], scene=Scene(children=(Group(children=(Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='green', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='yellow', emissiveMap=None, envMap=None, lightMap=None, map=None, side='BackSide', specularMap=None), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), DirectionalLight(intensity=0.6, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), AmbientLight(intensity=0.5, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap())

In [14]:
spin_track = NumberKeyframeTrack(name='.rotation[y]', times=[0, 2], values=[0, 6.28])
spin_clip = AnimationClip(tracks=[spin_track])
spin_action = AnimationAction(AnimationMixer(surf), spin_clip, surf)
spin_action

/home/loic/miniconda3/envs/cling-benjamin/lib/python3.5/site-packages/ipydatawidgets/ndarray/serializers.py:62: UserWarning: Cannot serialize (u)int64 data, Javascript does not support it. Casting to (u)int32.
  warnings.warn('Cannot serialize (u)int64 data, Javascript does not support it. '


AnimationAction(clip=AnimationClip(tracks=(NumberKeyframeTrack(name='.rotation[y]', times=array([0, 2]), values=array([0.  , 6.28], dtype=float32)),)), localRoot=Group(children=(Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16, type='ParametricGeometry'), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='green', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None, type='MeshLambertMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0)), Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16, type='ParametricGeometry'), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='yellow', emissiveMap=None, envMap=None, lightMap=None, map=None, side='BackSide', specularMap=None, type='MeshLambertMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0))), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Group', up=(0.0, 1.0, 0.0)), mixer=AnimationMixer(rootObject=Group(children=(Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16, type='ParametricGeometry'), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='green', emissiveMap=None, envMap=None, lightMap=None, map=None, specularMap=None, type='MeshLambertMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0)), Mesh(geometry=ParametricGeometry(func='\nfunction f(origu,origv) {\n    // scale u and v to the ranges I want: [0, 2*pi]\n    var u = 2*Math.PI*origu;\n    var v = 2*Math.PI*origv;\n    \n    var x = Math.sin(u);\n    var y = Math.cos(v);\n    var z = Math.cos(u+v);\n    \n    return new THREE.Vector3(x,y,z)\n}\n', slices=16, stacks=16, type='ParametricGeometry'), material=MeshLambertMaterial(alphaMap=None, aoMap=None, color='yellow', emissiveMap=None, envMap=None, lightMap=None, map=None, side='BackSide', specularMap=None, type='MeshLambertMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0))), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Group', up=(0.0, 1.0, 0.0))))

Note that we are spinning the object itself, and that we are therefore free to manipulate the camera at will.

## Morph targets

Set up a simple sphere geometry, and add a morph target that is an oblong pill shape:

In [15]:
# This lets three.js create the geometry, then syncs back vertex positions etc.
# For this reason, you should allow for the sync to complete before executing the next cell
morph = BufferGeometry.from_geometry(SphereBufferGeometry(1, 32, 16))

In [16]:
import numpy as np

# Set up morph targets:
vertices = np.array(morph.attributes['position'].array)
for i in range(len(vertices)):
    if vertices[i, 0] > 0:
        vertices[i, 0] += 1
morph.morphAttributes = {'position': [
    BufferAttribute(vertices),
]}

morphMesh = Mesh(morph, MeshPhongMaterial(color='#ff3333', shininess=150, morphTargets=True))

Set up animation for going back and forth between the sphere and pill shape:

In [17]:
pill_track = NumberKeyframeTrack(name='.morphTargetInfluences[0]', times=[0, 1.5, 3], values=[0, 2.5, 0])
pill_clip = AnimationClip(tracks=[pill_track])
pill_action = AnimationAction(AnimationMixer(morphMesh), pill_clip, morphMesh)

In [18]:
camera3 = PerspectiveCamera( position=[5, 3, 5], aspect=view_width/view_height)
scene3 = Scene(children=[morphMesh, camera3,
                         DirectionalLight(position=[3, 5, 1], intensity=0.6),
                         AmbientLight(intensity=0.5)])
renderer3 = Renderer(camera=camera3, scene=scene3, controls=[OrbitControls(controlling=camera3)],
                     width=view_width, height=view_height)
display(renderer3, pill_action)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(5.0, 3.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), controls=[OrbitControls(controlling=PerspectiveCamera(aspect=1.5, position=(5.0, 3.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)))], scene=Scene(children=(Mesh(geometry=BufferGeometry(attributes={'uv': BufferAttribute(array=array([[0.     , 1.     ],
       [0.03125, 1.     ],
       [0.0625 , 1.     ],
       ...,
       [0.9375 , 0.     ],
       [0.96875, 0.     ],
       [1.     , 0.     ]], dtype=float32), normalized=False, version=0), 'position': BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), normalized=False, version=0), 'normal': BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), normalized=False, version=0)}, morphAttributes={'position': (BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), version=1),)}, type='BufferGeometry'), material=MeshPhongMaterial(alphaMap=None, aoMap=None, bumpMap=None, color='#ff3333', displacementMap=None, emissiveMap=None, envMap=None, lightMap=None, map=None, morphTargets=True, normalMap=None, normalScale=(1.0, 1.0), shininess=150.0, specularMap=None, type='MeshPhongMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0)), PerspectiveCamera(aspect=1.5, position=(5.0, 3.0, 5.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), DirectionalLight(intensity=0.6, position=(3.0, 5.0, 1.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), AmbientLight(intensity=0.5, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0))), fog=None, overrideMaterial=None, quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), shadowMap=WebGLShadowMap())

AnimationAction(clip=AnimationClip(duration=3.0, tracks=(NumberKeyframeTrack(name='.morphTargetInfluences[0]', times=array([0. , 1.5, 3. ], dtype=float32), values=array([0. , 2.5, 0. ], dtype=float32)),)), localRoot=Mesh(geometry=BufferGeometry(attributes={'uv': BufferAttribute(array=array([[0.     , 1.     ],
       [0.03125, 1.     ],
       [0.0625 , 1.     ],
       ...,
       [0.9375 , 0.     ],
       [0.96875, 0.     ],
       [1.     , 0.     ]], dtype=float32), normalized=False, version=0), 'position': BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), normalized=False, version=0), 'normal': BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), normalized=False, version=0)}, morphAttributes={'position': (BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), version=1),)}, type='BufferGeometry'), material=MeshPhongMaterial(alphaMap=None, aoMap=None, bumpMap=None, color='#ff3333', displacementMap=None, emissiveMap=None, envMap=None, lightMap=None, map=None, morphTargets=True, normalMap=None, normalScale=(1.0, 1.0), shininess=150.0, specularMap=None, type='MeshPhongMaterial'), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Mesh', up=(0.0, 1.0, 0.0)), mixer=AnimationMixer(rootObject=Mesh(geometry=BufferGeometry(attributes={'uv': BufferAttribute(array=array([[0.     , 1.     ],
       [0.03125, 1.     ],
       [0.0625 , 1.     ],
       ...,
       [0.9375 , 0.     ],
       [0.96875, 0.     ],
       [1.     , 0.     ]], dtype=float32), normalized=False, version=0), 'position': BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), normalized=False, version=0), 'normal': BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), normalized=False, version=0)}, morphAttributes={'position': (BufferAttribute(array=array([[-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       [-0.0000000e+00,  1.0000000e+00,  0.0000000e+00],
       ...,
       [-1.1314261e-16, -1.0000000e+00, -4.6865205e-17],
       [-1.2011156e-16, -1.0000000e+00, -2.3891674e-17],
       [-1.2246469e-16, -1.0000000e+00, -2.9995195e-32]], dtype=float32), version=1),)}, type='BufferGeometry'), material=MeshPhongMaterial(alphaMap=None, aoMap=None, bumpMap=None, color='#ff3333', displacementMap=None, emissiveMap=None, envMap=None

## Skeletal animation

First, set up a skinned mesh with some bones:

In [19]:
import numpy as np

N_BONES = 3

ref_cylinder = CylinderGeometry(5, 5, 50, 5, N_BONES * 5, True)
cylinder = Geometry.from_geometry(ref_cylinder)

In [20]:
skinIndices = []
skinWeights = []

vertices = cylinder.vertices
boneHeight = ref_cylinder.height / (N_BONES - 1)
for i in range(len(vertices)):

    vertex = vertices[i];
    y = vertex[1] + 0.5 * ref_cylinder.height

    skinIndex = y // boneHeight;
    skinWeight = ( y % boneHeight ) / boneHeight;

    # Ease between each bone
    skinIndices.append([skinIndex, skinIndex + 1, 0, 0 ])
    skinWeights.append([1 - skinWeight, skinWeight, 0, 0 ])

cylinder.skinIndices = skinIndices
cylinder.skinWeights = skinWeights

shoulder = Bone(position=(0, -25, 0))
elbow = Bone(position=(0, 25, 0))
hand = Bone(position=(0, 25, 0))

shoulder.add(elbow)
elbow.add(hand)
bones = [shoulder, elbow, hand]
skeleton = Skeleton(bones)

mesh = SkinnedMesh(cylinder, MeshPhongMaterial(side='DoubleSide', skinning=True))
mesh.add(bones[0])
mesh.skeleton = skeleton

In [21]:
helper = SkeletonHelper(mesh)

Next, set up some simple rotation animations for the bones:

In [22]:
# Rotate on x and z axes:
bend_tracks = [
    NumberKeyframeTrack(name='.bones[1].rotation[x]', times=[0, 0.5, 1.5, 2], values=[0, 0.3, -0.3, 0]),
    NumberKeyframeTrack(name='.bones[1].rotation[z]', times=[0, 0.5, 1.5, 2], values=[0, 0.3, -0.3, 0]),
    NumberKeyframeTrack(name='.bones[2].rotation[x]', times=[0, 0.5, 1.5, 2], values=[0, -0.3, 0.3, 0]),
    NumberKeyframeTrack(name='.bones[2].rotation[z]', times=[0, 0.5, 1.5, 2], values=[0, -0.3, 0.3, 0]),
]
bend_clip = AnimationClip(tracks=bend_tracks)
bend_action = AnimationAction(AnimationMixer(mesh), bend_clip, mesh)
    
# Rotate on y axis:
wring_tracks = [
    NumberKeyframeTrack(name='.bones[1].rotation[y]', times=[0, 0.5, 1.5, 2], values=[0, 0.7, -0.7, 0]),
    NumberKeyframeTrack(name='.bones[2].rotation[y]', times=[0, 0.5, 1.5, 2], values=[0, 0.7, -0.7, 0]),
]

wring_clip = AnimationClip(tracks=wring_tracks)
wring_action = AnimationAction(AnimationMixer(mesh), wring_clip, mesh)

In [23]:
camera4 = PerspectiveCamera( position=[40, 24, 40], aspect=view_width/view_height)
scene4 = Scene(children=[mesh, helper, camera4,
                         DirectionalLight(position=[3, 5, 1], intensity=0.6),
                         AmbientLight(intensity=0.5)])
renderer4 = Renderer(camera=camera4, scene=scene4, controls=[OrbitControls(controlling=camera4)],
                     width=view_width, height=view_height)
display(renderer4)

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(40.0, 24.0, 40.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)), controls=[OrbitControls(controlling=PerspectiveCamera(aspect=1.5, position=(40.0, 24.0, 40.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), up=(0.0, 1.0, 0.0)))], scene=Scene(children=(SkinnedMesh(children=(Bone(children=(Bone(children=(Bone(position=(0.0, 25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), position=(0.0, 25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), position=(0.0, -25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), geometry=Geometry(faceVertexUvs=[[[{'x': 0, 'y': 1}, {'x': 0, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 1}], [{'x': 0, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 1}], [{'x': 0, 'y': 0.9333333373069763}, {'x': 0, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}], [{'x': 0, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}], [{'x': 0, 'y': 0.8666666746139526}, {'x': 0, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}], [{'x': 0, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}], [{'x': 0, 'y': 0.800000011920929}, {'x': 0, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.800000011920929}], [{'x': 0, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.800000011920929}], [{'x': 0, 'y': 0.7333333492279053}, {'x': 0, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}], [{'x': 0, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}], [{'x': 0, 'y': 0.6666666865348816}, {'x': 0, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}], [{'x': 0, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}], [{'x': 0, 'y': 0.6000000238418579}, {'x': 0, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}], [{'x': 0, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}], [{'x': 0, 'y': 0.5333333611488342}, {'x': 0, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}], [{'x': 0, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}], [{'x': 0, 'y': 0.46666666865348816}, {'x': 0, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}], [{'x': 0, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}], [{'x': 0, 'y': 0.4000000059604645}, {'x': 0, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}], [{'x': 0, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}], [{'x': 0, 'y': 0.3333333432674408}, {'x': 0, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}], [{'x': 0, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}], [{'x': 0, 'y': 0.2666666805744171}, {'x': 0, 'y': 0.20000000298023224}, {'x': 0.20000000298023224, 'y': 0.2666666805744171}], [{'x': 0, 'y': 0.20000000298023224}, {'x': 0.20000000298023224, 'y': 0.20000000298023224}, {'x': 0.20000000298023224, 'y': 0.2666666805744171}], [{'x': 0, 'y': 0.20000000298023224}

In [24]:
bend_action

AnimationAction(clip=AnimationClip(duration=2.0, tracks=(NumberKeyframeTrack(name='.bones[1].rotation[x]', times=array([0. , 0.5, 1.5, 2. ], dtype=float32), values=array([ 0. ,  0.3, -0.3,  0. ], dtype=float32)), NumberKeyframeTrack(name='.bones[1].rotation[z]', times=array([0. , 0.5, 1.5, 2. ], dtype=float32), values=array([ 0. ,  0.3, -0.3,  0. ], dtype=float32)), NumberKeyframeTrack(name='.bones[2].rotation[x]', times=array([0. , 0.5, 1.5, 2. ], dtype=float32), values=array([ 0. , -0.3,  0.3,  0. ], dtype=float32)), NumberKeyframeTrack(name='.bones[2].rotation[z]', times=array([0. , 0.5, 1.5, 2. ], dtype=float32), values=array([ 0. , -0.3,  0.3,  0. ], dtype=float32)))), localRoot=SkinnedMesh(children=(Bone(children=(Bone(children=(Bone(position=(0.0, 25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), position=(0.0, 25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), position=(0.0, -25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), geometry=Geometry(faceVertexUvs=[[[{'x': 0, 'y': 1}, {'x': 0, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 1}], [{'x': 0, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 1}], [{'x': 0, 'y': 0.9333333373069763}, {'x': 0, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}], [{'x': 0, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}], [{'x': 0, 'y': 0.8666666746139526}, {'x': 0, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}], [{'x': 0, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}], [{'x': 0, 'y': 0.800000011920929}, {'x': 0, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.800000011920929}], [{'x': 0, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.800000011920929}], [{'x': 0, 'y': 0.7333333492279053}, {'x': 0, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}], [{'x': 0, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}], [{'x': 0, 'y': 0.6666666865348816}, {'x': 0, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}], [{'x': 0, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}], [{'x': 0, 'y': 0.6000000238418579}, {'x': 0, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}], [{'x': 0, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}], [{'x': 0, 'y': 0.5333333611488342}, {'x': 0, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}], [{'x': 0, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}], [{'x': 0, 'y': 0.46666666865348816}, {'x': 0, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}], [{'x': 0, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}], [{'x': 0, 'y': 0.4000000059604645}, {'x': 0, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}], [{'x': 0, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}], [{'x': 0, 'y': 0.3333333432674408}, {'x': 0, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}], [{'x': 0, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.2666666805744171}, {'x': 0.2000000

In [25]:
wring_action

AnimationAction(clip=AnimationClip(duration=2.0, tracks=(NumberKeyframeTrack(name='.bones[1].rotation[y]', times=array([0. , 0.5, 1.5, 2. ], dtype=float32), values=array([ 0. ,  0.7, -0.7,  0. ], dtype=float32)), NumberKeyframeTrack(name='.bones[2].rotation[y]', times=array([0. , 0.5, 1.5, 2. ], dtype=float32), values=array([ 0. ,  0.7, -0.7,  0. ], dtype=float32)))), localRoot=SkinnedMesh(children=(Bone(children=(Bone(children=(Bone(position=(0.0, 25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), position=(0.0, 25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), position=(0.0, -25.0, 0.0), quaternion=(0.0, 0.0, 0.0, 1.0), scale=(1.0, 1.0, 1.0), type='Bone', up=(0.0, 1.0, 0.0)),), geometry=Geometry(faceVertexUvs=[[[{'x': 0, 'y': 1}, {'x': 0, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 1}], [{'x': 0, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}, {'x': 0.20000000298023224, 'y': 1}], [{'x': 0, 'y': 0.9333333373069763}, {'x': 0, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}], [{'x': 0, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}, {'x': 0.20000000298023224, 'y': 0.9333333373069763}], [{'x': 0, 'y': 0.8666666746139526}, {'x': 0, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}], [{'x': 0, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.800000011920929}, {'x': 0.20000000298023224, 'y': 0.8666666746139526}], [{'x': 0, 'y': 0.800000011920929}, {'x': 0, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.800000011920929}], [{'x': 0, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}, {'x': 0.20000000298023224, 'y': 0.800000011920929}], [{'x': 0, 'y': 0.7333333492279053}, {'x': 0, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}], [{'x': 0, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}, {'x': 0.20000000298023224, 'y': 0.7333333492279053}], [{'x': 0, 'y': 0.6666666865348816}, {'x': 0, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}], [{'x': 0, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}, {'x': 0.20000000298023224, 'y': 0.6666666865348816}], [{'x': 0, 'y': 0.6000000238418579}, {'x': 0, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}], [{'x': 0, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}, {'x': 0.20000000298023224, 'y': 0.6000000238418579}], [{'x': 0, 'y': 0.5333333611488342}, {'x': 0, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}], [{'x': 0, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}, {'x': 0.20000000298023224, 'y': 0.5333333611488342}], [{'x': 0, 'y': 0.46666666865348816}, {'x': 0, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}], [{'x': 0, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}, {'x': 0.20000000298023224, 'y': 0.46666666865348816}], [{'x': 0, 'y': 0.4000000059604645}, {'x': 0, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}], [{'x': 0, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}, {'x': 0.20000000298023224, 'y': 0.4000000059604645}], [{'x': 0, 'y': 0.3333333432674408}, {'x': 0, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}], [{'x': 0, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.2666666805744171}, {'x': 0.20000000298023224, 'y': 0.3333333432674408}], [{'x': 0, 'y': 0.2666666805744171}, {'x': 0, 'y': 0.20000000298023224}, {'x': 0.20000000298023224, 'y': 0.2666666805744171}], [{'x': 0, 'y': 0.20000000298023224}, {'x': 0.20000000298023224, 'y': 0.20000000298023224}, {'x': 0.20000000298023224, 'y': 0.2666666805744171}], [{